In [18]:
# 数据预处理尝试

import numpy as np
import pandas as pd

arr = pd.read_excel("tmp.xlsx")
# print(arr.head(10))


# 取出dataframe的几列
df1 = arr[["bmh","zzllmc","wgymc","ywk1mc","ywk2mc","bkzydm"]]

# 使用其中的四个属性进行分组
df2 = df1.groupby(["zzllmc","wgymc","ywk1mc","ywk2mc"]).count()
df2.to_excel("classfier.xlsx")


# sub_DF=[arr[arr['wgymc'].isin([i])] for i in np.unique(arr['wgymc'])]
# print(df2.head())
# groupby后的数据要先转为dataframe，再进行索引重建，才可以保存为xlsx
# class_arr = pd.DataFrame(df1.groupby(["zzllmc","wgymc","ywk1mc","ywk2mc"]))
# class_arr.reset_index(inplace=True)
# class_arr.to_excel("res.xlsx")

In [2]:
# 按照外国语考点代码拆分数据

import numpy as np
import pandas as pd

arr = pd.read_excel("tmp.xlsx")
list1 = arr['wgykddm'].unique()
groups = arr.groupby("wgykddm")
for i in arr['wgykddm'].unique():
    groups.get_group(i).to_excel(str(i)+".xlsx")
# print(groups['wgykddm'])
    # 检查数量是否相等
# print(arr.shape)#(16383,32)
# sum = 0;
# for i in arr['wgykddm'].unique():
#     df1 = pd.read_excel(str(i)+".xlsx")
#     sum += df1.shape[0]

# print(sum)#16383

In [15]:
# 统计每个考点教室所能容纳的人数
groups1 = pd.read_excel("考场容量表.xls").groupby("zzllkddm")
print(groups1.apply(lambda x:x))
print(groups1.get_group(6103)["renshu"].tolist())

WARNING *** file size (267964) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
               zzllkddm zzllkcbh  renshu
zzllkddm                                
6103     0         6103        1      30
         1         6103       10      30
         2         6103      100      30
         3         6103      101      30
         4         6103      102      30
...                 ...      ...     ...
6160     5420      6160      103      30
         5421      6160      104      30
         5422      6160      105      30
         5423      6160      106      30
         5424      6160      107      16

[5425 rows x 3 columns]
[30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 29, 30, 30, 30, 3

In [3]:
# 统计每个考点有多少专业，以及每个专业有多少人，参考属性有["zzllmc","wgymc","ywk1mc","ywk2mc","bkdwdm"]

for i in arr['wgykddm'].unique():
    df1 = pd.read_excel(str(i)+".xlsx")

    # 使用其中的四个属性进行分组
    df2 = df1.groupby(["zzllmc","wgymc","ywk1mc","ywk2mc","bkdwdm"]).count()
    df2.to_excel("classfier"+str(i)+".xlsx")

# 结果里面bmh表示的是每个专业的人数

In [5]:
# 数据预处理、提取出每个考点，每个专业的人数
sums = 0
print(list1)
for i in list1:
    # .fillna(method="ffill")向下填充
    res = pd.read_excel("classfier"+str(i)+".xlsx").fillna(method="ffill")
    # dataframe按各专业人数一列排序，inplace原地排序，ascending升序
    # res.sort_values(by="bmh",inplace=True,ascending=False)
    # 取出各专业人数一列转为array
    # 若存在控制需要填充
    # res = res.fillna(res['bmh'].mean())
    StuNum = res[['bmh','zzllmc','wgymc','ywk1mc','ywk2mc']]
    np.savetxt("classfier"+str(i)+".txt",StuNum,fmt="%d,%s,%s,%s,%s",delimiter=" ")
    sums += res['bmh'].sum()

print(sums)

# classfierX.excel中的前四列可以映射到对应的专业

[6116 6152 6103 6107 6146 6144 6143 6139 6117 6153 6118 6160 6141 6114
 6113 6124 6134 6105 6136 6149 6159 6157 6151 6115 6112 6111 6156 6133
 6148 6140 6125 6154 6147 6150 6145 6127 6138 6128 6122 6129]
16383


In [6]:
def Sum_matrix(matrix):
    sum=0
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            sum+=matrix[i][j]
    return sum

In [16]:
# 分组算法
# pd.set_option('display.max_rows',200)
# StuNum = np.loadtxt("classfier6127.txt")
aa=pd.read_csv("classfier6103.txt",encoding='gb2312',header=None)
aa.columns=['num','s1','s2','s3','s4']
# 对txt文件的数据排序，方便后面索引对应
print(aa.sort_values(by='num',axis=0).reset_index(drop=True))
StuNum = pd.read_csv("classfier6103.txt",encoding='gb2312',header=None).iloc[:,0].tolist()

# 设置参数：教室最多可容纳人数、考场允许最大专业数量
max_student=groups1.get_group(6103)["renshu"].tolist()
max_subject=5
# 存放分组结果
groups = []

# 这里还没有考虑取余的情况
for i in range(len(StuNum)):
    StuNum[i] = StuNum[i] % 30

StuNum.sort()
print(len(StuNum))
start = 0
end = len(StuNum)-1
classroom = 0
while(start<end and classroom<len(max_student)):
    class_subject = []#用来存储人数对应的索引，输出的就是这个，和上面txt文件的输出对应
    class_sum = StuNum[end]#每次循环初始，让sum等于尾指针指向的数
    subject = 1
    class_subject.append(end)
    # 人数不超过教室容量、专业不超过教室容量、专业还未分完
    while(class_sum<max_student[classroom] and subject<max_subject and start<end):
        class_sum = class_sum + StuNum[start]
        class_subject.append(start)
        start = start+1
        subject = subject+1
    end = end-1
    groups.append(class_subject)

print(groups)
print(len(groups))
# 下面的数字是不同袋子的编号

     num      s1     s2      s3       s4
0      1  思想政治理论     俄语  创意写作基础  文学评论与创作
1      1  思想政治理论  英语（二）   数学（三）    金融学综合
2      1  思想政治理论  英语（二）   数学（三）    金融学综合
3      1  思想政治理论  英语（二）   数学（三）      统计学
4      1  思想政治理论  英语（二）   数学（三）   税务专业基础
..   ...     ...    ...     ...      ...
152    9  思想政治理论  英语（一）   数学（二）     化工原理
153    9  思想政治理论  英语（一）   数学（二）   地球科学概论
154   11  思想政治理论  英语（二）   数学（二）       光学
155   14  思想政治理论  英语（二）   数学（二）     电路基础
156   26  思想政治理论  英语（二）   数学（二）    管理运筹学

[157 rows x 5 columns]
157
[[156, 0, 1, 2, 3], [155, 4, 5, 6, 7], [154, 8, 9, 10, 11], [153, 12, 13, 14, 15], [152, 16, 17, 18, 19], [151, 20, 21, 22, 23], [150, 24, 25, 26, 27], [149, 28, 29, 30, 31], [148, 32, 33, 34, 35], [147, 36, 37, 38, 39], [146, 40, 41, 42, 43], [145, 44, 45, 46, 47], [144, 48, 49, 50, 51], [143, 52, 53, 54, 55], [142, 56, 57, 58, 59], [141, 60, 61, 62, 63], [140, 64, 65, 66, 67], [139, 68, 69, 70, 71], [138, 72, 73, 74, 75], [137, 76, 77, 78, 79], [136, 80, 81, 82, 83], [135, 

In [ ]:
# 分组算法
import os
list1 = []
names = os.listdir("./")
for name in names:
    if name.endswith('.txt'):
        list1.append(name)
print(list1)

summ=0
for j in list1:
    # 读取文件并定位到人数一列
    if j=='classfier6127.txt':
        print(j)
        # header=None表示没有列标签
    StuNum = pd.read_csv(j,encoding='gb2312',header=None).iloc[:,0]
    # print(StuNum)

    # 设置参数：教室最多可容纳人数、考场允许最大专业数量
    max_student=30
    max_subject=5
    groups = []
    dic = {}
    # 建立原始值 和 余数的映射
    for i in range(len(StuNum)):
        dic[StuNum[i]] = StuNum[i] % 30

    # 保存字典映射关系
    StuNum = []
    with open('params.txt', 'w') as f:
        for key, value in dic.items():
            f.write(str(key))
            f.write(': ')
            f.write(str(value))
            f.write('\n')
            StuNum.append(value)


    # 对余数进行排序
    # dic = sorted(dic.items(),key=lambda x:x[1],reverse=False)
    StuNum.sort()
    # print(len(StuNum))
    start = 0
    end = len(StuNum)-1
    while(start<end):
        class_subject = []
        class_sum = StuNum[end]
        subject = 1
        class_subject.append(end)
        # ===================================================文件6127在这里出了问题
        while(class_sum<max_student and subject<max_subject):
            class_sum = class_sum + StuNum[start]
            class_subject.append(start)
            start = start+1
            subject = subject+1
        end = end-1
        groups.append(class_subject)

    print(groups)
    print(j)
    summ=summ+Sum_matrix(groups)
    print("\n")
print(summ)
    # 下面的数字是不同袋子的编号